In [ ]:
from pop import PiezoBuzzer, Touch, Led, PwmController, Textlcd
import time
import socket

# ---------------- 핀/설정 ----------------
PIN_BUZZER = 12
PIN_STATUS_LED = 27 
lcd = Textlcd()

word = "Hello AIoT Home"

pwm = PwmController()
pwm.init()
pwm.setChannel(4)

JAVA_SERVER_IP   = "192.168.101.102"
JAVA_SERVER_PORT = 39188

# ---------------- pop 장치 객체 ----------------
touch = Touch()
p = PiezoBuzzer(PIN_BUZZER)
p.setTempo(180)
leds = Led(PIN_STATUS_LED)

# ---------------- 도어락/상태 변수 ----------------
password   = [10, 2, 10, 4]   # 실제 키패드 번호 기준
input_pw   = []
unlocked   = False
fail_count = 0

# ---------------- 공통 함수들 ----------------
def send_event_to_java(event: str):
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.settimeout(0.5)  # connect, sendall, recv 모두 timeout
        s.connect((JAVA_SERVER_IP, JAVA_SERVER_PORT))
        s.sendall((event + "\n").encode())
        s.close()
    except Exception as e:
        print("자바 서버 전송 실패:", e)


def play_success_sound():
    p.tone(5, 4, 10)
    p.piezo.stop()
    p.tone(5, 8, 10)
    p.piezo.stop()
    p.tone(5, 12, 10)
    p.piezo.stop()

def play_error_sound():
    p.tone(4, 1, 10)
    p.piezo.stop()
    p.tone(3, 9, 10)
    p.piezo.stop()


def play_warning_sound():
    p.tone(5, 2, 12)
    p.piezo.stop()
    p.tone(4, 8, 12)
    p.piezo.stop()
    p.tone(4, 4, 12)
    p.piezo.stop()

def led_locked():
    # 잠김 상태 표현 (예: 끈 상태 = 잠김)
    leds.off()

def led_unlocked():
    # 풀림 상태 표현 (예: 계속 켜짐)
    leds.on()

def led_warning_blink(times=5):
    for _ in range(times):
        leds.on()
        time.sleep(0.2)
        leds.off()
        time.sleep(0.2)

def unlock_door():
    global unlocked
    unlocked = True
    send_event_to_java("UNLOCKED")
    led_unlocked()
    pwm.setFreq(50)
    pwm.setDuty((0.6/20)*100)
    print("문 열림!")
    lcd.clear()
    lcd.print(word)
    time.sleep(1)
    lcd.displayShiftR()
    time.sleep(1)
    lcd.displayShiftL()
    time.sleep(1)
    lcd.displayOff()
    time.sleep(1)
    lcd.displayOn()
    time.sleep(4)
    lcd.displayOff()

def lock_door():
    global unlocked
    unlocked = False
    led_locked()
    pwm.setFreq(50)
    pwm.setDuty((1.6/20)*100)
    print("문 잠김")
    send_event_to_java("LOCKED")


# ---------------- 메인 루프 (비밀번호 입력) ----------------
lock_door()  # 시작은 잠금 상태

print("스마트 도어락 시작")

while True:
    for ch in range(12):
        val = touch.readChannel(ch)

        if val == 1:
            # 키 변환 (9는 *, 11은 #)
            if ch == 9:
                key = '*'
            elif ch == 11:
                key = '#'
            else:
                key = str(ch)

            print("입력:", "*" * (len(input_pw) + 1))

            # 키 입력 사운드
            p.piezo.stop()
            p.tone(4, 4, 6) 
            p.piezo.stop()
            input_pw.append(ch)

            # 비밀번호 길이 다 찼을 때
            if len(input_pw) == len(password):
                if input_pw == password:
                    print("비밀번호 일치!")
                    p.piezo.stop()
                    play_success_sound()
                    p.piezo.stop()
                    unlock_door()
                    time.sleep(2)
                    lock_door()
                else:
                    print("비밀번호 오류")
                    fail_count += 1
                    p.piezo.stop()
                    play_error_sound()
                    p.piezo.stop()
                    led_warning_blink(3)

                    if fail_count >= 3:
                        print("3회 오류! 3초 동안 잠금")
                        send_event_to_java("ALERT_FAIL_3")
                        time.sleep(3)
                        fail_count = 0

                input_pw = []

            time.sleep(0.3)
